In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import swifter
import spacy
#from pprint import pprint

In [2]:
DATA_PATH = '../data'
RAW_DATA_PATH = DATA_PATH + '/raw'
CLEAN_DATA_PATH = DATA_PATH + '/clean'

In [3]:
le_monde_df = pd.read_csv(RAW_DATA_PATH + '/le_monde.csv', header=None)
le_monde_df

,0,1
0,https://www.lemonde.fr/disparitions/article/20...,"Simon Fieschi, grièvement blessé pendant l’att..."
1,https://www.lemonde.fr/idees/article/2024/10/1...,« Il faut expliquer la différence entre les ef...
2,https://www.lemonde.fr/economie/article/2024/1...,Le périphérique a désormais son outil évaluate...
3,https://www.lemonde.fr/economie/article/2024/1...,"En Occitanie, les agriculteurs mécontents mult..."
4,https://www.lemonde.fr/les-decodeurs/article/2...,Présidentielle américaine 2024 : comment Donal...
...,...,...
8561,https://www.lemonde.fr/international/article/2...,Retrait de Joe Biden : les réactions internati...
8562,https://www.lemonde.fr/culture/article/2024/07...,"A l’heure du bilan, le Festival « off » d’Avig..."
8563,https://www.lemonde.fr/series-d-ete/article/20...,"Les sneakers, partenaires de glisse du hip-hop"
8564,https://www.lemonde.fr/musiques/article/2024/0...,Nos choix de festivals et concerts : guitare e...


In [4]:
datafiles = []
for path in os.listdir(RAW_DATA_PATH):
    # check if current path is a file
    if os.path.isfile(os.path.join(RAW_DATA_PATH, path)):
        datafiles.append(path)
print(datafiles)

['liberation.csv', 'huffpost_insolite.csv', 'ouest_france.csv', 'actu_orange_insolite.csv', 'europe1_insolite.csv', 'franceinfotv_bizarre.csv', 'le_monde.csv', 'la_depeche.csv', 'franceinfotv.csv', 'rfi.csv', 'demotivateur.csv', 'le_parisien.csv']


In [5]:
dfs = {}
for datafile in datafiles:
    dfs[datafile.replace('.csv', '')] = pd.read_csv(f'{RAW_DATA_PATH}/{datafile}', header=None)

In [6]:
dfs['liberation']

,0,1
0,https://www.liberation.fr/international/ameriq...,"A New York, la mairie veut rendre la Ve Avenue..."
1,https://www.liberation.fr/forums/vers-une-resp...,Vers une responsabilité sociale et environneme...
2,https://www.liberation.fr/international/ameriq...,Présidentielle américaine 2024 : à la frontièr...
3,https://www.liberation.fr/forums/mal-logement-...,Mal-logement : «Les luttes des habitants sont ...
4,https://www.liberation.fr/culture/musique/waru...,"Warum Joe, punk canal historique"
...,...,...
5994,https://www.liberation.fr/international/ameriq...,«Kamala Harris part comme outsider face à Trum...
5995,https://www.liberation.fr/international/ameriq...,Tentative d’assassinat contre Trump : la direc...
5996,https://www.liberation.fr/sciences/decouverte-...,«Oxygène noir» : la molécule produite au fond ...
5997,https://www.liberation.fr/sports/jeux-olympiqu...,JO de Paris 2024 : l’escrimeuse Ysaora Thibus ...


In [7]:
for k, v in dfs.items():
    print(k, len(v))

liberation 5999
huffpost_insolite 10317
ouest_france 100233
actu_orange_insolite 600
europe1_insolite 1704
franceinfotv_bizarre 299
le_monde 8566
la_depeche 49322
franceinfotv 13977
rfi 7247
demotivateur 2381
le_parisien 17996


In [8]:
len(dfs['la_depeche']) // 3

16440

In [9]:
len(dfs['ouest_france']) // 6

16705

In [10]:
dfs['la_depeche'][:16440]

,0,1
0,https://www.ladepeche.fr/2024/10/19/bon-plan-b...,Bon plan bricolage : profitez du prix sacrifié...
1,https://www.ladepeche.fr/2024/10/19/enfin-de-r...,"Enfin de retour en stock, ce souffleur de feui..."
2,https://www.ladepeche.fr/2024/10/19/foncez-sur...,Foncez sur Cdiscount pour obtenir cette élague...
3,https://www.ladepeche.fr/2024/10/19/voici-la-m...,Voici la meilleure offre pour acheter le sécat...
4,https://www.ladepeche.fr/2024/10/19/bon-plan-a...,Bon plan aspirateur : 100 euros de remise immé...
...,...,...
16435,https://www.ladepeche.fr/2024/09/23/200-person...,200 personnes au repas du village
16436,https://www.ladepeche.fr/2024/09/23/de-ladour-...,De l’Adour à l’Arros
16437,https://www.ladepeche.fr/2024/09/23/du-cote-de...,Du côté de l’amicale loisirs et culture...
16438,https://www.ladepeche.fr/2024/09/23/festimages...,Fest’images : remise des portraits de l’expo


In [11]:
dfs['ouest_france'][:16705]

,0,1
0,https://www.ouest-france.fr/monde/gaza/33-mort...,33 morts dans une frappe israélienne sur un c...
1,https://www.ouest-france.fr/societe/faits-dive...,Policier blessé par arme à feu près de Nant...
2,https://www.ouest-france.fr/monde/japon/japon-...,Japon : un homme jette des cocktails molotov s...
3,https://www.ouest-france.fr/nouvelle-aquitaine...,Champdeniers-Saint-Denis. Aménagement de l’ac...
4,https://www.ouest-france.fr/nouvelle-aquitaine...,"Saint-Varent. Méthanisation, oui à l’extension"
...,...,...
16700,https://www.ouest-france.fr/pays-de-la-loire/t...,Le défi Foyers à alimentation positive d’Erd...
16701,https://www.ouest-france.fr/sport/football/red...,"Football. Le propriétaire du Red Star, 777 Pa..."
16702,https://www.ouest-france.fr/normandie/lisieux-...,"Plus d’un an après les émeutes à Lisieux, l..."
16703,https://www.ouest-france.fr/bretagne/finistere...,Le Sud-Finistère célèbre la culture urbaine...


In [12]:
dfs['ouest_france'][0][16704]

'https://www.ouest-france.fr/normandie/bretteville-sur-odon-14760/pres-de-caen-la-banque-alimentaire-recherche-des-benevoles-f6c3e4a6-840d-11ef-aa60-e9ecd5d09830'

In [13]:
dfs['la_depeche'] = dfs['la_depeche'][:16440]

In [14]:
dfs['ouest_france'] = dfs['ouest_france'][:16705]

In [15]:
for k, v in dfs.items():
    print(k, len(v))

liberation 5999
huffpost_insolite 10317
ouest_france 16705
actu_orange_insolite 600
europe1_insolite 1704
franceinfotv_bizarre 299
le_monde 8566
la_depeche 16440
franceinfotv 13977
rfi 7247
demotivateur 2381
le_parisien 17996


In [16]:
for k, v in dfs.items():
    v.insert(0, 'name', k)

In [17]:
dfs['le_monde']

,name,0,1
0,le_monde,https://www.lemonde.fr/disparitions/article/20...,"Simon Fieschi, grièvement blessé pendant l’att..."
1,le_monde,https://www.lemonde.fr/idees/article/2024/10/1...,« Il faut expliquer la différence entre les ef...
2,le_monde,https://www.lemonde.fr/economie/article/2024/1...,Le périphérique a désormais son outil évaluate...
3,le_monde,https://www.lemonde.fr/economie/article/2024/1...,"En Occitanie, les agriculteurs mécontents mult..."
4,le_monde,https://www.lemonde.fr/les-decodeurs/article/2...,Présidentielle américaine 2024 : comment Donal...
...,...,...,...
8561,le_monde,https://www.lemonde.fr/international/article/2...,Retrait de Joe Biden : les réactions internati...
8562,le_monde,https://www.lemonde.fr/culture/article/2024/07...,"A l’heure du bilan, le Festival « off » d’Avig..."
8563,le_monde,https://www.lemonde.fr/series-d-ete/article/20...,"Les sneakers, partenaires de glisse du hip-hop"
8564,le_monde,https://www.lemonde.fr/musiques/article/2024/0...,Nos choix de festivals et concerts : guitare e...


In [18]:
dfs['le_monde'][~dfs['le_monde'][0].str.contains(r"/\d+/\d+/\d+/", regex=True)]

,name,0,1
852,le_monde,https://fait-pour-durer.lemonde.fr/les-jeux-ol...,"Les Jeux Olympiques et Paralympiques, créateur..."
854,le_monde,https://fait-pour-durer.lemonde.fr/apres-les-j...,Jeux Olympiques et Paralympiques : après l’ent...
855,le_monde,https://fait-pour-durer.lemonde.fr/apres-les-j...,"Après les Jeux, des transports pour tous !"
856,le_monde,https://fait-pour-durer.lemonde.fr/les-sites-o...,Les Sites Olympiques au service des Franciliens
857,le_monde,https://reinventez-la-mobilite-avec-audi.lemon...,Une certaine idée du mouvement
969,le_monde,https://grand-seiko.lemonde.fr/gardien-du-temp...,"Grand Seiko, gardien du temps et de la tradition"
1235,le_monde,https://ruinart-renouveau-adresse.lemonde.fr/,"Ruinart, le renouveau d’une adresse emblématique"
2376,le_monde,https://hurtigruten.lemonde.fr/,Les croisières Hurtigruten : se connecter au c...
2613,le_monde,https://vivez-montreal-en-hiver.lemonde.fr/le-...,"Montréal, le temps des gourmands"
2614,le_monde,https://vivez-montreal-en-hiver.lemonde.fr/un-...,"Montréal, un hiver éblouissant"


In [19]:
df = pd.concat([df for df in dfs.values()]).reset_index(drop=True)

In [20]:
df

,name,0,1
0,liberation,https://www.liberation.fr/international/ameriq...,"A New York, la mairie veut rendre la Ve Avenue..."
1,liberation,https://www.liberation.fr/forums/vers-une-resp...,Vers une responsabilité sociale et environneme...
2,liberation,https://www.liberation.fr/international/ameriq...,Présidentielle américaine 2024 : à la frontièr...
3,liberation,https://www.liberation.fr/forums/mal-logement-...,Mal-logement : «Les luttes des habitants sont ...
4,liberation,https://www.liberation.fr/culture/musique/waru...,"Warum Joe, punk canal historique"
...,...,...,...
102226,le_parisien,https://www.leparisien.fr/international/etats-...,« Nous avons désormais une chance de gagner »...
102227,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : pourquoi le tournoi ..."
102228,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : quels sont les pays ..."
102229,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : le palmarès complet..."


In [21]:
df.columns = ['name', 'url', 'title']
df

,name,url,title
0,liberation,https://www.liberation.fr/international/ameriq...,"A New York, la mairie veut rendre la Ve Avenue..."
1,liberation,https://www.liberation.fr/forums/vers-une-resp...,Vers une responsabilité sociale et environneme...
2,liberation,https://www.liberation.fr/international/ameriq...,Présidentielle américaine 2024 : à la frontièr...
3,liberation,https://www.liberation.fr/forums/mal-logement-...,Mal-logement : «Les luttes des habitants sont ...
4,liberation,https://www.liberation.fr/culture/musique/waru...,"Warum Joe, punk canal historique"
...,...,...,...
102226,le_parisien,https://www.leparisien.fr/international/etats-...,« Nous avons désormais une chance de gagner »...
102227,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : pourquoi le tournoi ..."
102228,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : quels sont les pays ..."
102229,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : le palmarès complet..."


In [22]:
df.name.unique()

array(['liberation', 'huffpost_insolite', 'ouest_france',
       'actu_orange_insolite', 'europe1_insolite', 'franceinfotv_bizarre',
       'le_monde', 'la_depeche', 'franceinfotv', 'rfi', 'demotivateur',
       'le_parisien'], dtype=object)

In [23]:
df[df.url.duplicated()]

,name,url,title
6502,huffpost_insolite,https://www.huffingtonpost.fr/insolite/video/u...,Un python birman de six mètres capturé en Fl...
6508,huffpost_insolite,https://www.huffingtonpost.fr/insolite/video/u...,Un très rare « phallus de Titan » a fleuri en...
6557,huffpost_insolite,https://www.huffingtonpost.fr/culture/article/...,"Pour incarner Henri VIII, Jude Law s’est mis a..."
6563,huffpost_insolite,https://www.huffingtonpost.fr/culture/article/...,Que se passe-t-il avec les peintures de Pierre...
7761,huffpost_insolite,https://www.huffingtonpost.fr/culture/video/qu...,Quignon ou croûton: comment appelez-vous l'ex...
...,...,...,...
74513,franceinfotv,https://www.francetvinfo.fr/sports/sport-et-lo...,Vacances d’été : les dangers des jeux nautique...
74514,franceinfotv,https://www.francetvinfo.fr/economie/pouvoir-d...,Pouvoir d’achat : les robes de mariage d'occas...
74572,franceinfotv,https://www.francetvinfo.fr/les-jeux-olympique...,"REPORTAGE. ""Cela fait un an qu'on organise les..."
74573,franceinfotv,https://www.francetvinfo.fr/monde/usa/presiden...,"RÉCIT. Débat raté, lapsus en série et retrait...."


In [24]:
for index, row in df[df.url.duplicated()].sort_values('url').iterrows():
    print(row['name'], row['url'])

franceinfotv https://www.francetvinfo.fr/alpes-maritimes/alpes-maritimes-des-attaques-de-loups-dans-un-village_6803107.html
franceinfotv https://www.francetvinfo.fr/coupe-du-monde/masculine/coupe-du-monde-de-futsal-l-equipe-de-france-se-hisse-en-demi-finale-en-battant-le-paraguay_6810403.html
franceinfotv https://www.francetvinfo.fr/culture/arts-expos/street-art/une-chevre-des-elephants-et-des-singes-des-enigmatiques-banksy-animaliers-fleurissent-a-londres_6714042.html
franceinfotv https://www.francetvinfo.fr/culture/cinema/alain-delon/mort-d-alain-delon-de-romy-schneider-a-mireille-darc-qui-furent-les-femmes-de-sa-vie-sans-qui-il-n-aurait-ete-que-l-ombre-de-l-acteur-et-de-l-homme_3702783.html
franceinfotv https://www.francetvinfo.fr/culture/cinema/alain-delon/mort-d-alain-delon-film-comique-cap-de-charcuterie-l-homme-meconnu_6732897.html
franceinfotv https://www.francetvinfo.fr/culture/cinema/l-equipe-du-film-l-abbe-pierre-une-vie-de-combats-condamne-les-agissements-du-religieux-et-ap

In [25]:
df[df.url=='https://www.francetvinfo.fr/decouverte/bizarre/le-camping-des-eurockeennes-elit-son-maire_1620667.html']

,name,url,title
35533,franceinfotv_bizarre,https://www.francetvinfo.fr/decouverte/bizarre...,Le camping des Eurockéennes élit son maire
35534,franceinfotv_bizarre,https://www.francetvinfo.fr/decouverte/bizarre...,Le camping des Eurockéennes élit son maire


In [26]:
df = df.drop_duplicates('url').reset_index(drop=True)
df

,name,url,title
0,liberation,https://www.liberation.fr/international/ameriq...,"A New York, la mairie veut rendre la Ve Avenue..."
1,liberation,https://www.liberation.fr/forums/vers-une-resp...,Vers une responsabilité sociale et environneme...
2,liberation,https://www.liberation.fr/international/ameriq...,Présidentielle américaine 2024 : à la frontièr...
3,liberation,https://www.liberation.fr/forums/mal-logement-...,Mal-logement : «Les luttes des habitants sont ...
4,liberation,https://www.liberation.fr/culture/musique/waru...,"Warum Joe, punk canal historique"
...,...,...,...
102080,le_parisien,https://www.leparisien.fr/international/etats-...,« Nous avons désormais une chance de gagner »...
102081,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : pourquoi le tournoi ..."
102082,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : quels sont les pays ..."
102083,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : le palmarès complet..."


In [27]:
nlp = spacy.load("fr_core_news_md")

In [28]:
def lemmatize(s: str) -> iter:
    # tokenize
    doc = nlp(s)

    # remove punct and stopwords
    tokens = filter(lambda token: not token.is_space and not token.is_punct and not token.is_stop and not token.is_digit, doc)

    # lemmatize
    return map(lambda token: token.lemma_.lower(), tokens)

In [29]:
df['lemmes'] = df.title.swifter.apply(lambda text: ' '.join(map(str, lemmatize(text))))

Pandas Apply:   0%|          | 0/102085 [00:00<?, ?it/s]

In [30]:
df

,name,url,title,lemmes
0,liberation,https://www.liberation.fr/international/ameriq...,"A New York, la mairie veut rendre la Ve Avenue...",new york mairie vouloir ve avenue piéton
1,liberation,https://www.liberation.fr/forums/vers-une-resp...,Vers une responsabilité sociale et environneme...,responsabilité social environnemental musée
2,liberation,https://www.liberation.fr/international/ameriq...,Présidentielle américaine 2024 : à la frontièr...,présidentielle américain frontière mexicain es...
3,liberation,https://www.liberation.fr/forums/mal-logement-...,Mal-logement : «Les luttes des habitants sont ...,mal-logement lutte habitant revendication écol...
4,liberation,https://www.liberation.fr/culture/musique/waru...,"Warum Joe, punk canal historique",warum joe punk canal historique
...,...,...,...,...
102080,le_parisien,https://www.leparisien.fr/international/etats-...,« Nous avons désormais une chance de gagner »...,désormais chance gagner après retrait biden ...
102081,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : pourquoi le tournoi ...",jo paris handball tournoi terminer à lille à...
102082,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : quels sont les pays ...",jo paris handball pays titrés palmarès tourn...
102083,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : le palmarès complet...",jo paris handball palmarès complet équipe fr...


In [31]:
df['funny'] = 0
df

,name,url,title,lemmes,funny
0,liberation,https://www.liberation.fr/international/ameriq...,"A New York, la mairie veut rendre la Ve Avenue...",new york mairie vouloir ve avenue piéton,0
1,liberation,https://www.liberation.fr/forums/vers-une-resp...,Vers une responsabilité sociale et environneme...,responsabilité social environnemental musée,0
2,liberation,https://www.liberation.fr/international/ameriq...,Présidentielle américaine 2024 : à la frontièr...,présidentielle américain frontière mexicain es...,0
3,liberation,https://www.liberation.fr/forums/mal-logement-...,Mal-logement : «Les luttes des habitants sont ...,mal-logement lutte habitant revendication écol...,0
4,liberation,https://www.liberation.fr/culture/musique/waru...,"Warum Joe, punk canal historique",warum joe punk canal historique,0
...,...,...,...,...,...
102080,le_parisien,https://www.leparisien.fr/international/etats-...,« Nous avons désormais une chance de gagner »...,désormais chance gagner après retrait biden ...,0
102081,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : pourquoi le tournoi ...",jo paris handball tournoi terminer à lille à...,0
102082,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : quels sont les pays ...",jo paris handball pays titrés palmarès tourn...,0
102083,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : le palmarès complet...",jo paris handball palmarès complet équipe fr...,0


In [32]:
df[df['name'].isin(['actu_orange_insolite', 'demotivateur', 'europe1_insolite', 'franceinfotv_bizarre', 'huffpost_insolite'])]

,name,url,title,lemmes,funny
5999,huffpost_insolite,https://www.huffingtonpost.fr/insolite/video/u...,Un python birman de six mètres capturé en Fl...,python birman mètre capturé floride jamais,0
6000,huffpost_insolite,https://www.huffingtonpost.fr/insolite/video/u...,Un très rare « phallus de Titan » a fleuri en...,trè rare phallus titan fleurir france spectac...,0
6001,huffpost_insolite,https://www.huffingtonpost.fr/culture/article/...,"Pour incarner Henri VIII, Jude Law s’est mis a...",incarner henri viii jude law mettre à vraimen...,0
6002,huffpost_insolite,https://www.huffingtonpost.fr/culture/article/...,Que se passe-t-il avec les peintures de Pierre...,passer -t il peinture pierre soulages mettre a...,0
6003,huffpost_insolite,https://www.huffingtonpost.fr/culture/video/qu...,Quignon ou croûton: comment appelez-vous l'ex...,quignon croûton appeler vous extrémité pain,0
...,...,...,...,...,...
84084,demotivateur,https://www.demotivateur.fr/insolite/pour-l-ou...,"À Détroit, Eminem assure lui-même le service p...",détroit eminem assurer service ouverture resta...,0
84085,demotivateur,https://www.demotivateur.fr/insolite/scandinav...,Scandinavie : s'enfoncer des sachets de tabac ...,scandinavie enfoncer sachet tabac prépuce anus...,0
84086,demotivateur,https://www.demotivateur.fr/insolite/las-vegas...,"Découvrez les premières images de « Moon », l'...",découvrir premier image moon incroyable hôtel ...,0
84087,demotivateur,https://www.demotivateur.fr/insolite/ivre-et-a...,"Ivre et porté disparu, il est retrouvé par has...",ivre porter disparaître retrouver hasard parti...,0


In [33]:
df.loc[df['name'].isin(['actu_orange_insolite', 'demotivateur', 'europe1_insolite', 'franceinfotv_bizarre', 'huffpost_insolite']), 'funny'] = 1

In [34]:
df[df['name'].isin(['actu_orange_insolite', 'demotivateur', 'europe1_insolite', 'franceinfotv_bizarre', 'huffpost_insolite'])]

,name,url,title,lemmes,funny
5999,huffpost_insolite,https://www.huffingtonpost.fr/insolite/video/u...,Un python birman de six mètres capturé en Fl...,python birman mètre capturé floride jamais,1
6000,huffpost_insolite,https://www.huffingtonpost.fr/insolite/video/u...,Un très rare « phallus de Titan » a fleuri en...,trè rare phallus titan fleurir france spectac...,1
6001,huffpost_insolite,https://www.huffingtonpost.fr/culture/article/...,"Pour incarner Henri VIII, Jude Law s’est mis a...",incarner henri viii jude law mettre à vraimen...,1
6002,huffpost_insolite,https://www.huffingtonpost.fr/culture/article/...,Que se passe-t-il avec les peintures de Pierre...,passer -t il peinture pierre soulages mettre a...,1
6003,huffpost_insolite,https://www.huffingtonpost.fr/culture/video/qu...,Quignon ou croûton: comment appelez-vous l'ex...,quignon croûton appeler vous extrémité pain,1
...,...,...,...,...,...
84084,demotivateur,https://www.demotivateur.fr/insolite/pour-l-ou...,"À Détroit, Eminem assure lui-même le service p...",détroit eminem assurer service ouverture resta...,1
84085,demotivateur,https://www.demotivateur.fr/insolite/scandinav...,Scandinavie : s'enfoncer des sachets de tabac ...,scandinavie enfoncer sachet tabac prépuce anus...,1
84086,demotivateur,https://www.demotivateur.fr/insolite/las-vegas...,"Découvrez les premières images de « Moon », l'...",découvrir premier image moon incroyable hôtel ...,1
84087,demotivateur,https://www.demotivateur.fr/insolite/ivre-et-a...,"Ivre et porté disparu, il est retrouvé par has...",ivre porter disparaître retrouver hasard parti...,1


In [35]:
df[df['funny'] == 1].name.unique()

array(['huffpost_insolite', 'actu_orange_insolite', 'europe1_insolite',
       'franceinfotv_bizarre', 'demotivateur'], dtype=object)

In [36]:
df[df['funny'] == 0].name.unique()

array(['liberation', 'ouest_france', 'le_monde', 'la_depeche',
       'franceinfotv', 'rfi', 'le_parisien'], dtype=object)

In [37]:
df[df['name'].str.contains('_insolite')]

,name,url,title,lemmes,funny
5999,huffpost_insolite,https://www.huffingtonpost.fr/insolite/video/u...,Un python birman de six mètres capturé en Fl...,python birman mètre capturé floride jamais,1
6000,huffpost_insolite,https://www.huffingtonpost.fr/insolite/video/u...,Un très rare « phallus de Titan » a fleuri en...,trè rare phallus titan fleurir france spectac...,1
6001,huffpost_insolite,https://www.huffingtonpost.fr/culture/article/...,"Pour incarner Henri VIII, Jude Law s’est mis a...",incarner henri viii jude law mettre à vraimen...,1
6002,huffpost_insolite,https://www.huffingtonpost.fr/culture/article/...,Que se passe-t-il avec les peintures de Pierre...,passer -t il peinture pierre soulages mettre a...,1
6003,huffpost_insolite,https://www.huffingtonpost.fr/culture/video/qu...,Quignon ou croûton: comment appelez-vous l'ex...,quignon croûton appeler vous extrémité pain,1
...,...,...,...,...,...
35309,europe1_insolite,https://www.europe1.fr/insolite/La-Thailande-s...,La Thaïlande sauve 750 chiens de la cuisine,thaïlande sauf chien cuisine,1
35310,europe1_insolite,https://www.europe1.fr/insolite/Elle-transport...,Elle transportait de la cocaïne dans ses seins,transporter cocaïne sein,1
35311,europe1_insolite,https://www.europe1.fr/insolite/Intouchables-e...,Intouchables : elle accouche presque de rire,intouchable accouche presque rire,1
35312,europe1_insolite,https://www.europe1.fr/insolite/GB-un-cambriol...,GB : un cambrioleur insulte ses victimes,gb cambrioleur insulter victime,1


In [38]:
df['name'] = df['name'].str.replace('_insolite', '')

In [39]:
df['name'] = df['name'].str.replace('_bizarre', '')

In [40]:
df

,name,url,title,lemmes,funny
0,liberation,https://www.liberation.fr/international/ameriq...,"A New York, la mairie veut rendre la Ve Avenue...",new york mairie vouloir ve avenue piéton,0
1,liberation,https://www.liberation.fr/forums/vers-une-resp...,Vers une responsabilité sociale et environneme...,responsabilité social environnemental musée,0
2,liberation,https://www.liberation.fr/international/ameriq...,Présidentielle américaine 2024 : à la frontièr...,présidentielle américain frontière mexicain es...,0
3,liberation,https://www.liberation.fr/forums/mal-logement-...,Mal-logement : «Les luttes des habitants sont ...,mal-logement lutte habitant revendication écol...,0
4,liberation,https://www.liberation.fr/culture/musique/waru...,"Warum Joe, punk canal historique",warum joe punk canal historique,0
...,...,...,...,...,...
102080,le_parisien,https://www.leparisien.fr/international/etats-...,« Nous avons désormais une chance de gagner »...,désormais chance gagner après retrait biden ...,0
102081,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : pourquoi le tournoi ...",jo paris handball tournoi terminer à lille à...,0
102082,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : quels sont les pays ...",jo paris handball pays titrés palmarès tourn...,0
102083,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : le palmarès complet...",jo paris handball palmarès complet équipe fr...,0


In [41]:
df.name.unique()

array(['liberation', 'huffpost', 'ouest_france', 'actu_orange', 'europe1',
       'franceinfotv', 'le_monde', 'la_depeche', 'rfi', 'demotivateur',
       'le_parisien'], dtype=object)

In [42]:
print(df[['name', 'funny']].drop_duplicates().sort_values('funny').to_string(index=False))

        name  funny
  liberation      0
ouest_france      0
    le_monde      0
  la_depeche      0
         rfi      0
 le_parisien      0
franceinfotv      0
 actu_orange      1
     europe1      1
franceinfotv      1
    huffpost      1
demotivateur      1


In [43]:
df.to_csv(CLEAN_DATA_PATH + '/dataset.csv', index=False)